# 本セクションの目次

# IoT/Kafka/Spark Streamingを組み合わせてみよう

今回は、IoTとしてWebブラウザを利用してユーザのウェブ画面上での操作をトラッキングしてみたいと思います。

Web画面には3つの機能があります
1. ログイン(/でアクセスするとログインしたということにします)　http://nodejs:3000
2. カートに入れる(/cart)　http://nodejs:3000/cart
3. 決済する(/done) http://nodejs:3000/done

ユーザがこれらのアクションを取ったらkafkaへデータが飛んでくるような仕組みになっています。

実際の現場では、これらのデータをもとに分析を行い対策を練っていきます。  
ex. カートにまで入るけど決済されないなぁ



In [ ]:
# コンソールで設定したSparkとNoteBookを接続します(動かす前に毎度実行する必要があります)
import findspark
findspark.init("/home/pyspark/spark")

In [ ]:
#pysparkに必要なライブラリを読み込む
from pyspark import SparkConf
from pyspark import SparkContext
from pyspark.sql import SparkSession

#spark sessionの作成
# spark.ui.enabled trueとするとSparkのGUI画面を確認することができます
# spark.eventLog.enabled true　とすると　GUIで実行ログを確認することができます
# GUIなどの確認は最後のセクションで説明を行います。
spark = SparkSession.builder \
    .appName("chapter1") \
    .config("hive.exec.dynamic.partition", "true") \
    .config("hive.exec.dynamic.partition.mode", "nonstrict") \
    .config("spark.sql.session.timeZone", "JST") \
    .config("spark.ui.enabled","true") \
    .config("spark.eventLog.enabled","true") \
    .config("spark.jars.packages", "org.apache.spark:spark-streaming_2.13:3.2.0,org.apache.spark:spark-sql-kafka-0-10_2.12:3.2.0,org.apache.spark:spark-avro_2.12:3.2.0") \
    .enableHiveSupport() \
    .getOrCreate()

# パッケージを複数渡したい時は「,」で繋いで渡します。
# Sparkのバージョンにしっかりと合わせます(今回はSparkのバージョンが3.2を使っています。)。

In [ ]:
# kafkaからデータを読み取る設定を行います。
# 今回はpyspark-topic3から読み取ります

df = spark \
  .readStream \
  .format("kafka") \
  .option("kafka.bootstrap.servers", "kafka:9092") \
  .trigger(processingTime="5 seconds") \
  .option("subscribe", "pyspark-topic3") \
  .load()

In [ ]:
memory_sink = df \
  .selectExpr("CAST(key AS STRING)", "CAST(value AS STRING)") \
  .writeStream \
  .format("memory") \
  .queryName("web_actions") \
  .trigger(processingTime="5 seconds") \
  .option("checkpointLocation", "/tmp/kafka/memory/") \
  .start()

-------------データをいくつか送信してみます---------------------